<a href="https://colab.research.google.com/github/LuigiPagani/ROM-with-POD-and-DL/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
     from dlroms import*
except:
     !pip install git+https://github.com/NicolaRFranco/dlroms.git
     from dlroms import*
!pip install pyDOE



  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18168 sha256=fcf0ca7370faa7baa5d8dd172575be6582689de5ccb6184b65e4b9890b96aaf5
  Stored in directory: /root/.cache/pip/wheels/ce/b6/d7/c6b64746dba6433c593e471e0ac3acf4f36040456d1d160d17
Successfully built pyDOE


**Preliminaries**

In [112]:
import numpy as np
from fenics import *
from pyDOE import lhs
from scipy.sparse.linalg import spsolve
import dlroms.fespaces as fe
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
from convdiff import Stokes_solver
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

mesh = fe.unitsquaremesh(30, 30)
Vh = fe.space(mesh, 'CG', 1) # FE space for u (chemical concentration)
bspace = fe.space(mesh, "CG", 1, scalar = False, bubble = True) # FE space for b (transport field)

def Convdiff_solver(w1, w2, steps = 50, dt = 5e-4):
    # Assembling relevant operators
    v1, v2 = TrialFunction(Vh), TestFunction(Vh)
    M = sp.csr_matrix(assemble(v1*v2*dx).array()) # mass matrix
    S = sp.csr_matrix(assemble(inner(grad(v1), grad(v2))*dx).array()) # stiffness (diffusion) matrix

    b = Stokes_solver(w1, w2)
    b1 = Stokes_solver(1,0)
    b2 = Stokes_solver(0,1)
    bf = fe.asvector(b, bspace)
    bf1 = fe.asvector(b1, bspace)
    bf2 = fe.asvector(b2, bspace)

    B = sp.csr_matrix(assemble(inner(bf, grad(v1))*v2*dx).array()) # transport matrix
    B1 = sp.csr_matrix(assemble(inner(bf1, grad(v1))*v2*dx).array())
    B2 = sp.csr_matrix(assemble(inner(bf2, grad(v1))*v2*dx).array())

    # Time-stepping scheme
    def FOMstep(u0, dt, b):
        A = M + dt*S + 1000*dt*B
        F = M @ u0
        return spsolve(A, F)

    # Initial condition
    x, y = fe.coordinates(Vh).T
    u0 = np.exp(-16*(x-0.5)**2 -16*(y-0.5)**2)

    # Initialize storage for solution as a NumPy array
    u = np.empty((len(u0), steps + 1))
    u[:, 0] = u0

    # Time loop
    for n in range(steps):
        u[:, n + 1] = FOMstep(u[:, n], dt, b)

    return u, M, S, B, B1, B2


# Define the parameter space
n_samples = 10  # Reduced number of samples
parameter_ranges = [np.linspace(0, 30, 100), np.linspace(0, 30, 100)]  # Define parameter ranges

# Generate Latin Hypercube Samples
lhs_samples = lhs(2, samples=n_samples, criterion='center')  # 2 for two parameters, criterion 'center' for centered samples
params = np.array([parameter_ranges[0][((lhs_samples[:, i] * 99).astype(int))] for i in range(2)]).T

# Generate snapshots
snapshots = []  # Initialize an empty list to store matrices
for w1, w2 in params:
    u, M, S, B,B1,B2 = Convdiff_solver(w1, w2)  # Get the time series data and matrices from the solver
    u = u.reshape(961, -1)  # Reshape u to have consistent dimensions
    if len(snapshots) == 0:
        snapshots = u  # If snapshots is empty, initialize it with the first u
    else:
        snapshots = np.hstack((snapshots, u))  # Stack u horizontally with the existing snapshots




**Matrix Assembly**

In [113]:
from scipy.linalg import svd

U, s, Vt = svd(snapshots, full_matrices=False)


# Now you can apply SVD or any other analysis you wish to this giant matrix
from scipy.linalg import svd
U, s, Vt = svd(snapshots, full_matrices=False)

energy = np.cumsum(s**2) / np.sum(s**2)

# Determine number of modes to retain 99% of energy
n_modes = np.searchsorted(energy, 0.999999) + 1

V = U[:, :n_modes]



mesh = fe.unitsquaremesh(30, 30)
Vh = fe.space(mesh, 'CG', 1) # FE space for u (chemical concentration)
bspace = fe.space(mesh, "CG", 1, scalar = False, bubble = True) # FE space for b (transport field)

from scipy.sparse import csr_matrix

# Assuming we have a function to assemble FOM matrices
w1 = 1
w2 = 2
u, M_fom, S_fom, B_fom,B1_fom,B2_fom = Convdiff_solver(w1, w2, steps = 50, dt = 5e-4)

M_rb = V.T @ M_fom @ V
S_rb = V.T @ S_fom @ V
B_rb = V.T @ B_fom @ V
B1_rb = V.T @ B1_fom @ V
B2_rb = V.T @ B2_fom @ V

# Calculate the linear combination of B1 and B2 using the given weights
B_combined_fom = w1 * B1_fom + w2 * B2_fom

# Compute the difference between B_fom and the weighted linear combination
difference_fom = B_fom - B_combined_fom
print(V.shape)




(961, 49)


In [108]:
print(s)

[1.46435235e+02 4.63712841e+01 3.21702309e+01 2.64790911e+01
 1.11966278e+01 1.02081124e+01 7.87057845e+00 6.07281150e+00
 4.54761802e+00 3.39895466e+00 3.31275873e+00 2.50103348e+00
 2.42106003e+00 2.07318654e+00 1.81479267e+00 1.41203777e+00
 1.17045129e+00 9.68678271e-01 7.70352605e-01 6.86766401e-01
 5.91176882e-01 5.33049675e-01 4.84519279e-01 3.87568061e-01
 3.26351802e-01 3.15746743e-01 2.50572097e-01 2.30551071e-01
 2.20551902e-01 1.97336004e-01 1.77914920e-01 1.53168367e-01
 1.46729478e-01 1.19353724e-01 1.03485912e-01 9.28738195e-02
 8.57750149e-02 8.02739169e-02 6.66450187e-02 6.07229744e-02
 5.11863637e-02 4.61878634e-02 4.25187930e-02 4.00707190e-02
 3.51123183e-02 3.08478566e-02 2.86481034e-02 2.42834632e-02
 2.26675768e-02 2.13294583e-02 1.78432182e-02 1.62662857e-02
 1.51946210e-02 1.22521311e-02 1.13103565e-02 1.12245770e-02
 8.97959216e-03 8.35557514e-03 7.44320766e-03 6.50502254e-03
 6.37906885e-03 5.02388085e-03 4.98376434e-03 3.64119438e-03
 3.40300488e-03 3.131620

**Time Stepping**
Fixes Pending


In [114]:

def RBstep(u_rb, dt):
    # Assuming M_rb, S_rb, B1_rb, B2_rb, w1, w2 are defined elsewhere and are numpy arrays
    A_rb = M_rb + dt * S_rb + 1000 * dt * (w1 * B1_rb + w2 * B2_rb)
    F_rb = M_rb @ u_rb
    return np.linalg.solve(A_rb, F_rb)

# Assuming 'V' is the matrix containing the left singular vectors (reduced basis vectors)
# and 'u' contains the simulation snapshots from the Full Order Model (FOM).

u0_rb = V.T @ u[:,0]  # Project the initial condition onto the reduced basis

# Number of time steps based on final time T and time step dt
T_final = 0.025
dt = 5e-4
n_steps = int(T_final / dt)

# Initialize storage for RB solution as a NumPy array
u_rb = np.zeros((V.shape[1], n_steps + 1))  # V.shape[1] is the number of reduced basis vectors
u_rb[:, 0] = u0_rb

# Time-stepping loop for the RB method
for n in range(n_steps):
    u_rb[:, n + 1] = RBstep(u_rb[:, n], dt)

# Optionally, convert back to full space for visualization or further analysis
u_rb_full = V @ u_rb  # This results in each column representing the solution at a time step

# If you need to transpose for further processing:

In [110]:
print(V.shape,u_rb.shape, u_rb_full.shape,u.shape)

(961, 38) (38, 51) (961, 51) (961, 51)


**Error Calculation**

In [115]:

# Assuming 'u' is the original full matrix, and 'u_rb_full' is the reconstructed matrix after reduction and back-projection

error_vector = u - u_rb_full

# Calculate the norm of the difference (numerator of the relative error)
error_norm = np.linalg.norm(error_vector, axis=0)  # Since we transposed, use axis=0 to operate along columns

# Calculate the norm of the original vector (denominator of the relative error)
u_norm = np.linalg.norm(u, axis=0)  # Norm of each column in the original matrix

# Calculate the relative error for each column
relative_error = error_norm / u_norm

# Calculate the average of the relative errors
average_relative_error = np.mean(relative_error)

print("Average Relative Error:", average_relative_error)


Average Relative Error: 0.05389834704491816
